In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import confusion_matrix
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3830)])
    except RuntimeError as e:
        print(e)

2024-05-15 15:00:22.413871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 15:00:22.417500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 15:00:22.683490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 15:00:23.831805: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages

In [2]:
# Load train, validation and test datasets
train, train_info = tfds.load(name='beans', split='train', shuffle_files=True, with_info=True, as_supervised=True)
valid, valid_info = tfds.load(name='beans', split='validation', shuffle_files=False, with_info=True, as_supervised=True)
test, test_info = tfds.load(name='beans', split='test', shuffle_files=False, with_info=True, as_supervised=True)

2024-05-15 15:00:36.653687: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 15:00:36.653763: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 15:00:36.653789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 15:00:37.194710: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-15 15:00:37.194736: I tensorflow/core/common_runtime/gpu/gpu

In [3]:
class printCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_epoch_end(self, epoch, logs=None):
                if (int(epoch) % 10) == 0:
                    print(
                        f"Epoch: {epoch:>3}"
                        + f" | Loss: {logs['loss']:.4e}"
                        + f" | Accuracy: {logs['accuracy']:.4e}\n"
                        + f" | Validation loss: {logs['val_loss']:.4e}"
                        + f" | Validation accuracy: {logs['val_accuracy']:.4e}")

In [4]:
image_size = (180, 180)

resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(image_size[0], image_size[1], interpolation='bicubic'),
    tf.keras.layers.Normalization()
])

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation([-1, 1]),
    tf.keras.layers.RandomContrast(0.33),
    tf.keras.layers.RandomBrightness(0.2),
])

In [6]:
batch_size = 64

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y))

    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(batch_size)

    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x), y))

    return ds.cache().repeat().prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
train_a = prepare(train, shuffle=False, augment=False)
valid_a = prepare(valid)
test_a = prepare(test)

In [7]:
resnet = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
resnet.trainable = False

model = tf.keras.Sequential([
    resnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(train_a,
        epochs=40,
        steps_per_epoch=len(train)//batch_size,
        validation_data=valid_a,
        validation_steps=len(valid)//batch_size,
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(factor=0.74, patience=3, min_lr=0.0001, verbose=True)], #printCallback()],
        #verbose=0
        )

Epoch 1/40


2024-05-08 11:04:43.989034: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2024-05-08 11:04:49.357441: I external/local_xla/xla/service/service.cc:168] XLA service 0x557f57708310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-08 11:04:49.357486: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-05-08 11:04:49.362427: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1715166289.428385    8572 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16/16 [==============================] - 19s 704ms/step - loss: 1.2909 - accuracy: 0.5938 - val_loss: 0.6844 - val_accuracy: 0.7734 - lr: 0.0010
Epoch 2/40
16/16 [==============================] - 3s 143ms/step - loss: 0.6667 - accuracy: 0.7330 - val_loss: 0.4809 - val_accuracy: 0.8281 - lr: 0.0010
Epoch 3/40
16/16 [==============================] - 2s 151ms/step - loss: 0.5002 - accuracy: 0.8021 - val_loss: 0.4429 - val_accuracy: 0.8359 - lr: 0.0010
Epoch 4/40
16/16 [==============================] - 2s 136ms/step - loss: 0.4351 - accuracy: 0.8351 - val_loss: 0.3839 - val_accuracy: 0.8594 - lr: 0.0010
Epoch 5/40
16/16 [==============================] - 2s 137ms/step - loss: 0.3938 - accuracy: 0.8392 - val_loss: 0.3743 - val_accuracy: 0.8359 - lr: 0.0010
Epoch 6/40
16/16 [==============================] - 2s 137ms/step - loss: 0.3388 - accuracy: 0.8680 - val_loss: 0.3243 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 7/40
16/16 [==============================] - 2s 136ms/step - loss: 0.3174

In [19]:
test_loss, test_accuracy = model.evaluate(test_a, steps=len(test)//batch_size)
tested = test_a.take(10)
test_labels = np.array([sample["label"] for sample in tested])
y_pred = np.argmax(model.predict(tested), axis=-1)
print("Loss:", test_loss, "Accuracy:", test_accuracy)
conf_matrix = confusion_matrix(test_labels, y_pred)
print(conf_matrix)

10/10 [==============================] - 1s 119ms/step
Loss: 0.15984579920768738 Accuracy: 0.9296875
